# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I {**MUKTHAR**, **ABDULWAHEED**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [ ]:
pip install pyspark==3.0

In [1]:
pip install pydeequ

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

In [8]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [12]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1963
df = spark.read.parquet("parquet/1963/data_ingestion.parquet")

In [13]:
df.show(5)

+-------------------+----------+----------+---------+----------+-----------+--------+-----+
|               date|      open|      high|      low|     close|  adj_close|  volume|stock|
+-------------------+----------+----------+---------+----------+-----------+--------+-----+
|1963-10-28 00:00:00|  6.924645|  6.940665| 6.908625|  6.908625|  1.6844347| 39900.0|   AA|
|1963-10-28 00:00:00|  6.488943| 6.5123687| 6.465517|  6.465517|  1.5472991| 42600.0| ARNC|
|1963-10-28 00:00:00|0.56790125|0.56995887|0.5617284|0.56995887| 0.11004562|789600.0|   BA|
|1963-10-28 00:00:00| 1.8541666| 1.9166666|1.8541666| 1.9166666|  0.1730804|319200.0|  CAT|
|1963-10-28 00:00:00|       0.0| 4.0859375|4.0234375|  4.046875|0.067614794|156800.0|  CVX|
+-------------------+----------+----------+---------+----------+-----------+--------+-----+
only showing top 5 rows



In [14]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- stock: string (nullable = true)



In [16]:
meta_df = spark.read.csv('symbols_valid_meta.csv', header=True)
meta_df.show(5)

+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|Nasdaq Traded|Symbol|       Security Name|Listing Exchange|Market Category|ETF|Round Lot Size|Test Issue|Financial Status|CQS Symbol|NASDAQ Symbol|NextShares|
+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|            Y|     A|Agilent Technolog...|               N|               |  N|         100.0|         N|            null|         A|            A|         N|
|            Y|    AA|Alcoa Corporation...|               N|               |  N|         100.0|         N|            null|        AA|           AA|         N|
|            Y|  AAAU|Perth Mint Physic...|               P|               |  Y|         100.0|         N|            null|      AAAU|         AAAU|         N|
|            Y|  AACG|ATA Creativity Gl.

## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [17]:
#TODO: Write your code here
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Null value Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(  
     check.isComplete("date")\
    .isComplete("open")\
    .isComplete("high")\
    .isComplete("low")\
    .isComplete("close")\
    .isComplete("adj_close")\
    .isComplete("volume")\
    .isComplete("stock"))\
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+----------------+-----------+------------+----------------------------------------------------+-----------------+------------------+
|check           |check_level|check_status|constraint                                          |constraint_status|constraint_message|
+----------------+-----------+------------+----------------------------------------------------+-----------------+------------------+
|Null value Check|Warning    |Success     |CompletenessConstraint(Completeness(date,None))     |Success          |                  |
|Null value Check|Warning    |Success     |CompletenessConstraint(Completeness(open,None))     |Success          |                  |
|Null value Check|Warning    |Success     |CompletenessConstraint(Completeness(high,None))     |Success          |                  |
|Null value Check|Warning    |Success     |CompletenessConstraint(Completeness(low,None))      |Success          |                  |
|Null value Check|Warning    |Success     |CompletenessConstra

C:\ProgramData\Anaconda3\lib\site-packages\pyspark\sql\dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [18]:
#TODO: Write your code here
check = Check(spark, CheckLevel.Warning, "Zero value Check")
result = VerificationSuite(spark)\
.onData(df)\
.addCheck(check
    .satisfies("open == 0", "Zero value check", lambda x: x==0)\
    .satisfies("high == 0", "Zero value check", lambda x: x==0)\
    .satisfies("low == 0", "Zero value check", lambda x: x==0)\
    .satisfies("close == 0", "Zero value check", lambda x: x==0)\
    .satisfies("adj_close == 0", "Zero value check", lambda x: x==0)\
    .satisfies("volume == 0", "Zero value check", lambda x: x==0)\
    )\
.run()

result_df = VerificationResult.checkResultsAsDataFrame(spark, result)
result_df.show(truncate=False)

Python Callback server started!
+----------------+-----------+------------+----------------------------------------------------------------------+-----------------+--------------------------------------------------------------------+
|check           |check_level|check_status|constraint                                                            |constraint_status|constraint_message                                                  |
+----------------+-----------+------------+----------------------------------------------------------------------+-----------------+--------------------------------------------------------------------+
|Zero value Check|Warning    |Warning     |ComplianceConstraint(Compliance(Zero value check,open == 0,None))     |Failure          |Value: 0.549800796812749 does not meet the constraint requirement!  |
|Zero value Check|Warning    |Warning     |ComplianceConstraint(Compliance(Zero value check,high == 0,None))     |Success          |                            

## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [19]:
#TODO: Write your code here
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Negative value Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(  
     check.isNonNegative("date")\
    .isNonNegative("open")\
    .isNonNegative("high")\
    .isNonNegative("low")\
    .isNonNegative("close")\
    .isPositive("adj_close")\
    .isNonNegative("volume")\
    .isNonNegative("stock"))\
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+--------------------+-----------+------------+-----------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|check               |check_level|check_status|constraint                                                                                                       |constraint_status|constraint_message                                  |
+--------------------+-----------+------------+-----------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|Negative value Check|Warning    |Warning     |ComplianceConstraint(Compliance(date is non-negative,COALESCE(CAST(date AS DECIMAL(20,10)), 0.0) >= 0,None))     |Failure          |Value: 0.0 does not meet the constraint requirement!|
|Negative value Check|Warning    |Warning     |ComplianceConstraint(

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [24]:
#TODO: Write your code here
from pydeequ.profiles import *
result = ColumnProfilerRunner(spark) \
    .onData(df.select('open','high','low','close','adj_close','volume')) \
    .run()
for col, profile in result.profiles.items():
    print(f"Maximum value for {col} column is "+ str(profile.maximum))

## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [26]:
#TODO: Write your code here
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Stock ticker consistency Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(  
     check.isContainedIn("stock",np.array(meta_df.select('Symbol').collect()).reshape(-1))\
    )\
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)

In [27]:
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Stock ticker cons...|    Warning|     Success|ComplianceConstra...|          Success|                  |
+--------------------+-----------+------------+--------------------+-----------------+------------------+



## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [28]:
#TODO: Write your code here
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Duplication test Check")
checkResult = VerificationSuite(spark).onData(df).addCheck(check\
                                                          #  .isPrimaryKey(["date","stock"])\
                                                           .hasUniqueness(["date","stock"], lambda x:x ==1)\
                                                           .hasUniqueness(['open','high','low','close','adj_close','volume'], lambda x:x ==1)
                                                           ).run()
                                                           
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate= False)

+----------------------+-----------+------------+------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|check                 |check_level|check_status|constraint                                            |constraint_status|constraint_message                                                 |
+----------------------+-----------+------------+------------------------------------------------------+-----------------+-------------------------------------------------------------------+
|Duplication test Check|Warning    |Warning     |UniquenessConstraint(Uniqueness(Stream(date, ?),None))|Success          |                                                                   |
|Duplication test Check|Warning    |Warning     |UniquenessConstraint(Uniqueness(Stream(open, ?),None))|Failure          |Value: 0.9976095617529881 does not meet the constraint requirement!|
+----------------------+-----------+---------